<a href="https://colab.research.google.com/github/meashu/Pyspark_Archive_Dataset_Analayis/blob/main/json_archive_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=5294cfc8730ce8afac69ffd450ba5bb87fe3b0a570802631d9b7375d296f2c8a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark=SparkSession.builder.appName('Practice-DF').getOrCreate()


**### Working with Json ###**

In [23]:
! pip install kaggle

In [24]:
! mkdir ~/.kaggle

In [25]:
! cp kaggle.json ~/.kaggle/

In [26]:
! chmod 600 ~/.kaggle/kaggle.json

In [28]:
! kaggle datasets download Cornell-University/arxiv

100% 1.25G/1.25G [00:16<00:00, 52.8MB/s]
100% 1.25G/1.25G [00:16<00:00, 82.2MB/s]


In [29]:
! unzip /content/arxiv.zip

Archive:  /content/arxiv.zip
  inflating: arxiv-metadata-oai-snapshot.json  


In [30]:
df=spark.read.format('json').load('/content/arxiv-metadata-oai-snapshot.json')

In [32]:
df.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_parsed: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- categories: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- journal-ref: string (nullable = true)
 |-- license: string (nullable = true)
 |-- report-no: string (nullable = true)
 |-- submitter: string (nullable = true)
 |-- title: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)



In [34]:
#Checking Number of partitions
df.rdd.getNumPartitions(),

31

In [36]:
#Create a  new Schema

from pyspark.sql.types import *

Schema=StructType([
    StructField('abstract',StringType(),True),
    StructField('authors',StringType(),True),
    StructField('categories',StringType(),True),
    StructField('license',StringType(),True),
    StructField('comments',StringType(),True),
    StructField('versions',ArrayType(StringType()),True),
])

In [37]:
print(Schema)

StructType([StructField('abstract', StringType(), True), StructField('authors', StringType(), True), StructField('categories', StringType(), True), StructField('license', StringType(), True), StructField('comments', StringType(), True), StructField('versions', ArrayType(StringType(), True), True)])


In [38]:
#read data again with our schema

df=spark.read.format('json').load('/content/arxiv-metadata-oai-snapshot.json',schema=Schema,header=True)

In [39]:
df.show()

+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|            abstract|             authors|       categories|             license|            comments|            versions|
+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|           hep-ph|                NULL|37 pages, 15 figu...|[{"version":"v1",...|
|  We describe a n...|Ileana Streinu an...|    math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|[{"version":"v1",...|
|  The evolution o...|         Hongjun Pan|   physics.gen-ph|                NULL| 23 pages, 3 figures|[{"version":"v1",...|
|  We show that a ...|        David Callan|          math.CO|                NULL|            11 pages|[{"version":"v1",...|
|  In this paper w...|Wael Abu-Shammala...|  math.CA math.FA|                NULL|                NULL|[{"version":"v1",...|


In [41]:
#It will delete the values with null in comments
df=df.dropna(subset=['comments'])

#It will replace null with unknown for column license
df=df.fillna(value="unknown",subset=['license'])

In [42]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            abstract|             authors|          categories|             license|            comments|            versions|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|              hep-ph|             unknown|37 pages, 15 figu...|[{"version":"v1",...|
|  We describe a n...|Ileana Streinu an...|       math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|[{"version":"v1",...|
|  The evolution o...|         Hongjun Pan|      physics.gen-ph|             unknown| 23 pages, 3 figures|[{"version":"v1",...|
|  We show that a ...|        David Callan|             math.CO|             unknown|            11 pages|[{"version":"v1",...|
|  We study the tw...|Y. H. Pong and C....|   cond-mat.mes-hall|             unknown|6 pages, 4 figure..

In [49]:
#KPI 1-  Get author name who published a paper in a 'math' category

df.select(col('authors')).filter(col('categories').like('%math%')).show(truncate=False)


df.createOrReplaceTempView('Archive')
sql_query="""
              Select authors from Archive where categories like '%math%'
          """

result=spark.sql(sql_query)
result.show(truncate=False)


+--------------------+
|             authors|
+--------------------+
|Ileana Streinu an...|
|        David Callan|
|  Sergei Ovchinnikov|
|Clifton Cunningha...|
|        Koichi Fujii|
|         Norio Konno|
|Simon J.A. Malham...|
|Robert P. C. de M...|
|  P\'eter E. Frenkel|
|          Mihai Popa|
|   Debashish Goswami|
|I. Csiszar, F. Hi...|
|      Mikkel {\O}bro|
|Nabil L. Youssef,...|
|         Boris Rubin|
|         A. I. Molev|
| Branko J. Malesevic|
|   John W. Robertson|
|     Yu.N. Kosovtsov|
|        Osamu Fujino|
+--------------------+
only showing top 20 rows

+--------------------------------------------+
|authors                                     |
+--------------------------------------------+
|Ileana Streinu and Louis Theran             |
|David Callan                                |
|Sergei Ovchinnikov                          |
|Clifton Cunningham and Lassina Dembele      |
|Koichi Fujii                                |
|Norio Konno                                 |

In [51]:
###KPI-2  GET LICENSE WITH 5 OR MORE LETTERS IN THE "ABSTRACT"

sql_query="""
              select license from archive  where abstract  REGEXP '%\(([A-Za-z][^_ /\\<>$]{5,})\)%'

          """

result=spark.sql(sql_query).show(truncate=False)


+---------------------------------------------------+
|license                                            |
+---------------------------------------------------+
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://creativecommons.org/licenses/by/4.0/        |
|http://creativecommons.org/licenses/by/4.0/        |
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://arxiv.org/licenses/nonexclusive-distrib/1.0/|
|http://creativecommons.org/licenses/by-nc-sa/4.0/  |
|http://creativecommons.org/licenses/by-nc-nd/4.0/  |
|http://arxiv.org/licenses/n

In [55]:
###KPI-3  Extract the stats of the number of pages of unknown license
import re
def get_page(line):
  search=re.findall('\d+ pages',line)      ### \d+  search for number 1 or more , space,  after sapce pages'
  if search:
    return int(search[0].split(" ")[0])        #### split with space and pick only number
  else:
    return 0


In [58]:
spark.udf.register("get_page",get_page)

<function __main__.get_page(line)>

In [59]:
sql_query="""select avg(get_page(comments)) as avg ,sum(get_page(comments)) as sum, std(get_page(comments)) as std from archive where  license='unknown' """

result=spark.sql(sql_query).show()

+------------------+---------+------------------+
|               avg|      sum|               std|
+------------------+---------+------------------+
|13.365886832870077|5640578.0|16.775196504270717|
+------------------+---------+------------------+

